In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Parameter windowing
input_width = 60*2
label_width = 60
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
train_df = None


In [2]:
def  import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv'):
    
    df_begin=pd.read_csv(file_path)
    df_1=df_begin.copy()
    df_1.drop(columns=['RRU.MaxLayerDlMimo', 'CARR.AverageLayersDl','RRU.PrbAvailDl', 'DRB.UEThpUl','RRU.PrbAvailUl','RRU.PrbTotUl','RRU.PrbTotDl','RRC.ConnMax','RRU.PrbUsedUl'], inplace=True)
    convert_time=pd.to_datetime(df_1['timestamp'], unit='s',origin='unix')
    df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
    df_1.set_index('datetime_column', inplace=True)
    df_1.drop(columns=['timestamp'], inplace=True)
    
    corr_matrix = df_1.corr(numeric_only=True)[['RRU.PrbUsedDl']]
    filtered_corr = corr_matrix[['RRU.PrbUsedDl']]

    # Find columns where correlation with 'RRU.PrbUsedDl'is greater than 0.5
    columns_with_high_corr = filtered_corr.index[(filtered_corr['RRU.PrbUsedDl'] > 0.5) ].tolist()

    seleted_columns = ['Viavi.Cell.Name']+ columns_with_high_corr 
    df_2= df_1[seleted_columns].copy()
    cell_name= df_2['Viavi.Cell.Name'].unique()
    df= df_2.copy()
    
    return df, cell_name

In [3]:
def standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data - mean) / std

def inverse_standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data * std) + mean


In [4]:
def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)


In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

def tensorflow_cnn(X_train_scaled, Y_train_scaled, X_validation_scaled, Y_validation_scaled, 
                    learning_rate, target_error, max_epochs, max_sampel_batch,
                    patience, save_best_model_path, validation_data=False, load_model=None, out_steps=OUT_STEPS):
    global model

    class MAEStopCallback(tf.keras.callbacks.Callback):
        def __init__(self, threshold):
            super(MAEStopCallback, self).__init__()
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            if logs['mae'] < self.threshold:
                print(f"\nMAE reached below {self.threshold}. Stopping training.")
                self.model.stop_training = True

                
    input_width = X_train_scaled.shape[1]
    CONV_WIDTH = input_width # Define the width of the convolutional window
    num_features = X_train_scaled.shape[2]
    num_output = Y_train_scaled.shape[2]
    out_steps = out_steps

    model = tf.keras.models.Sequential()
    if load_model is None:
        model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :], input_shape=(input_width, num_features)))
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=CONV_WIDTH, padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH/2),padding='same') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH),padding='valid') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(out_steps * num_output, kernel_initializer=tf.initializers.zeros()))
        model.add(tf.keras.layers.Reshape([out_steps, num_output]))
        # model.add(tf.keras.layers.LSTM(units=120, input_shape=(input_width, num_features), return_sequences=True))
        # model.add(tf.keras.layers.LSTM(units=120, return_sequences=True))
        # model.add(tf.keras.layers.LSTM(units=120, return_sequences=True))
        # model.add(tf.keras.layers.LSTM(units=120, return_sequences=False))
        # model.add(tf.keras.layers.Dense(out_steps*num_output,   kernel_initializer=tf.initializers.zeros()))
        # model.add(tf.keras.layers.Reshape([out_steps, num_output]))
    else:
        model = tf.keras.models.load_model(load_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    mae_stop_callback = MAEStopCallback(threshold=target_error)
    checkpoint_callback = ModelCheckpoint(
        save_best_model_path,
        monitor='val_loss',     
        mode='min',         
        save_best_only=True, 
        verbose=1            
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',     
        mode='min',         
        patience=patience,    
        restore_best_weights=True,
        verbose=1            
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(2), min_lr=0.00001, verbose=1)

    time_start = time.time()
    if validation_data:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch,  
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr], 
                  validation_data=(X_validation_scaled, Y_validation_scaled), validation_batch_size=max_sampel_batch)
    else:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch, 
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr])
    
    print("time computation seconds: ", time.time() - time_start)
    
    loss, MSE, MAE, RMSE, MAPE = model.evaluate(X_train_scaled, Y_train_scaled)
    print("loss: ", loss, "MSE: ", MSE, "MAE: ", MAE, "RMSE: ", RMSE, "MAPE: ", MAPE)
    
    return model, loss, MSE, MAE, RMSE, MAPE


In [6]:
def compute_metrics(actual_df, predictions_df):
    mae = mean_absolute_error(actual_df, predictions_df)
    mse = mean_squared_error(actual_df, predictions_df)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual_df - predictions_df) / actual_df)) * 100
    return mae, mape, mse, rmse

def process_predictions(model, x_scaled, actual_df, input_width, label_width, columns):
    predictions = model.predict(x_scaled[slice(None, None, label_width), :, :])
    predictions_reshaped = predictions.reshape(-1, len(columns))
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    predictions_df = pd.DataFrame(predictions_unscaled, columns=columns, index=actual_df[input_width:].index)
    return predictions_df
def compute_error( x_scaled, actual_df, columns, model):
    
    # Process predictions for training and validation sets
    predictions_df = process_predictions(model, x_scaled, actual_df, input_width, label_width, columns)

    display(pd.concat((predictions_df.rename(columns={columns[0]: '%s_predict'%columns[0]}), actual_df[input_width:]), axis=1))
    mae, mape, mse, rmse = compute_metrics(actual_df[input_width:], predictions_df)
    
    print(f"MAE: {mae}, MAPE %: {mape}, MSE: {mse}, RMSE: {rmse}")

In [7]:

def plot_predictions(val_df, x_val_scaled, model, val_scaled, input_width, label_width, num_features):
    num_output = 2


    # Predictions for validation data
    val_predictions = model.predict(x_val_scaled[slice(None,None,label_width), :, :])
    predictions_reshaped = val_predictions.reshape(-1, num_output)
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    val_predictions_df = pd.DataFrame(predictions_unscaled, columns=['RRU.PrbUsedDl'], index=val_df.index)

    # Generate new predictions
    val_new = val_scaled.tail(input_width).to_numpy()
    val_new = val_new.reshape(1, -1, num_features)
    generate_predictions = model.predict(val_new)
    generate_predictions = inverse_standardize_data(generate_predictions.reshape(-1, num_output), isoutput=True, train_df=train_df)
    generate_predictions_df = pd.DataFrame(generate_predictions, columns=['RRU.PrbUsedDl'], index=val_df.tail(label_width).index + pd.DateOffset(hours=1))

    # Combine predictions
    all_predictions = pd.concat([val_predictions_df, generate_predictions_df], axis=0)
    inputs_graph = val_df[['RRU.PrbUsedDl']].iloc[-label_width * 3:-label_width]
    labels_graph = val_df[['RRU.PrbUsedDl']].iloc[-label_width:]
    predictions_graph = all_predictions.loc[labels_graph.index]
    new_predictions_graph = all_predictions.loc[generate_predictions_df.index]

    # Plotting
    plt.figure(figsize=(16, 8))
    for n, feature in enumerate(all_predictions.columns):
        plt.subplot(len(all_predictions.columns), 1, n + 1)
        plt.plot(inputs_graph.index, inputs_graph[feature], label='Inputs for orange', marker='.', zorder=-100, markersize=5)
        plt.plot(labels_graph.index, labels_graph[feature], label='actual output for orange and inputs for red', marker='.', zorder=-100, c='#2ca02c')
        plt.scatter(predictions_graph.index, predictions_graph[feature], marker='X', edgecolors='k', label='Prediction', c='#ff7f0e', s=int(32 * 1.5))
        plt.scatter(new_predictions_graph.index, new_predictions_graph[feature], marker='+', label='New prediction', c='#FF012D', s=int(32 * 1.5))
        plt.ylabel(feature)
        plt.legend()

    plt.subplots_adjust(hspace=0.2, top=1)
    plt.show()


In [8]:
def running_program(my_df,cell_name, index_cell, name_file, name_file_before):
    global train_df
    df_1= my_df[my_df['Viavi.Cell.Name']==cell_name[index_cell]].drop(columns=['Viavi.Cell.Name']).copy()
    df=df_1.astype(float).copy()
    n = len(df)
    train_df =df.loc[:'2023-01-07 00:00:00'].iloc[:-1]
    val_df = df['2023-01-07 00:00:00':]
    num_features = df.shape[1]
    train_scaled = standardize_data(train_df, isoutput=False, train_df=train_df)
    val_scaled= standardize_data(val_df, isoutput=False, train_df=train_df)
    # Membuat windowed dataset untuk set pelatihan, validasi, dan pengujian

    x_train_scaled, y_train_scaled = make_windows(train_scaled.to_numpy(), train_scaled[['RRU.PrbUsedDl']].to_numpy(),total_window_size, input_slice, labels_slice)
    x_val_scaled, y_val_scaled = make_windows(val_scaled.to_numpy(), val_scaled[['RRU.PrbUsedDl']].to_numpy(),total_window_size, input_slice, labels_slice)
    print(x_train_scaled.shape, y_train_scaled.shape)
    if index_cell==0:
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=20, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=None, out_steps=OUT_STEPS)
        model.summary()
        model.save(name_file)
    else:
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=20, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=name_file_before, out_steps=OUT_STEPS)
    
    model = tf.keras.models.load_model('%s'%name_file)
    columns = ['RRU.PrbUsedDl']
    output_actual_train = train_df[columns]
    output_actual_val = val_df[columns]
    print("Training Metrics:")
    compute_error(x_train_scaled,  output_actual_train,columns, model)
    print("\nValidation Metrics:")
    compute_error(x_val_scaled,  output_actual_val,columns, model)
    #print("Training Plot:")
    #plot_predictions( output_actual_train[input_width:], x_train_scaled, model, train_scaled, input_width, label_width, num_features)
    #print("\nValidation (test) Plot:")
    #plot_predictions( output_actual_val[input_width:], x_val_scaled, model, val_scaled, input_width, label_width, num_features)
    


In [9]:
df_start, cell_name= import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv')
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M")
skip_indices = [26,29, 31, 37, 40]
for index in range(0,52):
    print(index)
    if index in skip_indices:
        print(index)
        continue
    cells_name= df_start['Viavi.Cell.Name'].unique()
    df_filtered = df_start[df_start['Viavi.Cell.Name'] == cells_name[index]]
    df_no_duplicates = df_filtered.loc[~df_filtered.index.duplicated()]
    name_file='part4_%s_%s.hdf5'%(timestamp, index+1)
    name_file_before='part4_%s_%s.hdf5'%(timestamp, index)
    if index in np.array(skip_indices)+1:
        name_file_before='part4_%s_%s.hdf5'%(timestamp, index-1)
    print("name_file: ", name_file)
    print("name_file_before: ", name_file_before)
    print("Cell Name: ", cell_name[index])
    running_program(my_df=df_no_duplicates, cell_name=cell_name, index_cell=index, name_file=name_file, name_file_before=name_file_before)

0
name_file:  part4_20240729_1333_1.hdf5
name_file_before:  part4_20240729_1333_0.hdf5
Cell Name:  S1/B2/C1
(8941, 120, 6) (8941, 60, 1)
Epoch 1/20
150/150 [==============================] - ETA: 0s - loss: 0.7080 - mse: 0.7080 - mae: 0.5825 - mape: 210.7825 - rmse: 0.8414
Epoch 1: val_loss improved from inf to 0.93652, saving model to part4_20240729_1333_1.hdf5
150/150 [==============================] - 5s 14ms/step - loss: 0.7080 - mse: 0.7080 - mae: 0.5825 - mape: 210.7825 - rmse: 0.8414 - val_loss: 0.9365 - val_mse: 0.9365 - val_mae: 0.6028 - val_mape: 147.7699 - val_rmse: 0.9677 - lr: 0.0010
Epoch 2/20
146/150 [============================>.] - ETA: 0s - loss: 0.3401 - mse: 0.3401 - mae: 0.4238 - mape: 180.2803 - rmse: 0.5832
Epoch 2: val_loss did not improve from 0.93652
150/150 [==============================] - 2s 12ms/step - loss: 0.3364 - mse: 0.3364 - mae: 0.4213 - mape: 179.1617 - rmse: 0.5800 - val_loss: 1.4822 - val_mse: 1.4822 - val_mae: 0.9420 - val_mape: 386.6624 - val

,RRU.PrbUsedDl_predict,RRU.PrbUsedDl
datetime_column,,
2022-12-31 18:00:00,97.605676,98.333333
2022-12-31 18:01:00,101.014770,100.000000
2022-12-31 18:02:00,75.939205,75.083333
2022-12-31 18:03:00,101.166313,100.000000
2022-12-31 18:04:00,99.088952,100.000000
...,...,...
2023-01-06 23:55:00,99.478750,98.066667
2023-01-06 23:56:00,95.052683,97.233333
2023-01-06 23:57:00,93.294214,93.816667


MAE: 1.166507816177555, MAPE %: 1.596966335653723, MSE: 2.580348171394732, RMSE: 1.6063462177857961

Validation Metrics:
1/1 [==============================] - 0s 32ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedDl
datetime_column,,
2023-01-07 02:00:00,94.713084,93.950000
2023-01-07 02:01:00,97.109359,100.000000
2023-01-07 02:02:00,96.573575,96.650000
2023-01-07 02:03:00,84.670706,72.933333
2023-01-07 02:04:00,98.110068,100.000000
...,...,...
2023-01-07 15:55:00,83.914584,63.333333
2023-01-07 15:56:00,89.486058,100.000000
2023-01-07 15:57:00,85.737909,100.000000


MAE: 7.5317528354873025, MAPE %: 10.676512053011871, MSE: 123.91342614412179, RMSE: 11.131640766038123
1
name_file:  part4_20240729_1333_2.hdf5
name_file_before:  part4_20240729_1333_1.hdf5
Cell Name:  S7/B2/C1
(8941, 120, 6) (8941, 60, 1)
Epoch 1/20
147/150 [============================>.] - ETA: 0s - loss: 0.2723 - mse: 0.2723 - mae: 0.3876 - mape: 131.7580 - rmse: 0.5219
Epoch 1: val_loss improved from inf to 0.60016, saving model to part4_20240729_1333_2.hdf5
150/150 [==============================] - 3s 15ms/step - loss: 0.2696 - mse: 0.2696 - mae: 0.3851 - mape: 130.9599 - rmse: 0.5193 - val_loss: 0.6002 - val_mse: 0.6002 - val_mae: 0.6543 - val_mape: 197.4056 - val_rmse: 0.7747 - lr: 0.0010
Epoch 2/20
147/150 [============================>.] - ETA: 0s - loss: 0.0429 - mse: 0.0429 - mae: 0.1646 - mape: 61.1747 - rmse: 0.2071
Epoch 2: val_loss improved from 0.60016 to 0.52961, saving model to part4_20240729_1333_2.hdf5
150/150 [==============================] - 2s 14ms/step - loss

...

MAE: 48.38775379496829, MAPE %: inf, MSE: 3530.8167851922885, RMSE: 59.42067641143349
51
name_file:  part4_20240729_1333_52.hdf5
name_file_before:  part4_20240729_1333_51.hdf5
Cell Name:  S4/N77/C3
(8941, 120, 6) (8941, 60, 1)
Epoch 1/20
149/150 [============================>.] - ETA: 0s - loss: 0.3177 - mse: 0.3177 - mae: 0.4142 - mape: 127.9116 - rmse: 0.5637
Epoch 1: val_loss improved from inf to 0.63974, saving model to part4_20240729_1333_52.hdf5
150/150 [==============================] - 4s 20ms/step - loss: 0.3178 - mse: 0.3178 - mae: 0.4143 - mape: 127.9084 - rmse: 0.5638 - val_loss: 0.6397 - val_mse: 0.6397 - val_mae: 0.6362 - val_mape: 163.2225 - val_rmse: 0.7998 - lr: 0.0010
Epoch 2/20
148/150 [============================>.] - ETA: 0s - loss: 0.0471 - mse: 0.0471 - mae: 0.1710 - mape: 58.0961 - rmse: 0.2170
Epoch 2: val_loss did not improve from 0.63974
150/150 [==============================] - 3s 19ms/step - loss: 0.0471 - mse: 0.0471 - mae: 0.1709 - mape: 58.0635 - rmse:

,RRU.PrbUsedDl_predict,RRU.PrbUsedDl
datetime_column,,
2022-12-31 18:00:00,29.678448,22.75
2022-12-31 18:01:00,100.062557,100.10
2022-12-31 18:02:00,18.528965,18.20
2022-12-31 18:03:00,14.781234,18.20
2022-12-31 18:04:00,3.014741,0.00
...,...,...
2023-01-06 23:55:00,44.786944,45.50
2023-01-06 23:56:00,165.678436,168.35
2023-01-06 23:57:00,137.720152,136.50


MAE: 1.3288772795605996, MAPE %: inf, MSE: 2.7952878491376683, RMSE: 1.671911435793675

Validation Metrics:
1/1 [==============================] - 0s 19ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedDl
datetime_column,,
2023-01-07 02:00:00,-5.950645,0.00
2023-01-07 02:01:00,135.002445,195.65
2023-01-07 02:02:00,79.633218,145.60
2023-01-07 02:03:00,62.404009,27.30
2023-01-07 02:04:00,75.680735,95.55
...,...,...
2023-01-07 15:55:00,148.072913,163.80
2023-01-07 15:56:00,43.524420,13.65
2023-01-07 15:57:00,65.022624,136.50


MAE: 36.84732967098815, MAPE %: inf, MSE: 2261.398129133394, RMSE: 47.55415995613206
